In [1]:
import os, glob, hashlib
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

PERSIST_DIR = "./index/chroma"
COLLECTION = "artificial_intelligence"
EMBED = "intfloat/multilingual-e5-small"  # good for Finnish
CORPUS_DIR = "./corpus"

def load_docs(corpus_dir=CORPUS_DIR):
    paths = glob.glob(os.path.join(corpus_dir, "**", "*"), recursive=True)
    docs = []
    for p in paths:
        pl = p.lower()
        if pl.endswith(".pdf"):
            docs += PyPDFLoader(p).load()
        elif pl.endswith((".txt", ".md")):
            docs += TextLoader(p, encoding="utf-8").load()
    return docs

def make_id(text: str) -> str:
    # stable chunk id to avoid duplicates
    return hashlib.sha1(text.strip().encode("utf-8")).hexdigest()[:16]

def main():
    raw_docs = load_docs()
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=800, chunk_overlap=120
    )
    chunks = splitter.split_documents(raw_docs)

    emb = HuggingFaceEmbeddings(model_name=EMBED)

    vs = Chroma(
        collection_name=COLLECTION,
        persist_directory=PERSIST_DIR,
        embedding_function=emb,
    )

    texts, metas, ids = [], [], []
    for d in chunks:
        txt = d.page_content
        src = d.metadata.get("source", "unknown")
        page = d.metadata.get("page", None)
        cid = make_id(f"{src}-{page}-{txt[:200]}")
        texts.append(txt)
        metas.append({"source": src, "page": page})
        ids.append(cid)

    # upsert is idempotent if ids repeat
    vs.add_texts(texts=texts, metadatas=metas, ids=ids)

    # make sure it’s persisted
    vs.persist()
    print(f"Indexed {len(texts)} chunks into '{COLLECTION}' at {PERSIST_DIR}")

if __name__ == "__main__":
    main()


C:\Users\jaakk\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\jaakk\AppData\Local\Temp\ipykernel_15544\4248334748.py:34: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  emb = HuggingFaceEmbeddings(model_name=EMBED)
C:\Users\jaakk\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not s

Indexed 508 chunks into 'artificial_intelligence' at ./index/chroma


C:\Users\jaakk\AppData\Local\Temp\ipykernel_15544\4248334748.py:56: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vs.persist()


['q', 'w', 'e', 'r']
